In [1]:
import pandas as pd
import numpy as np
import warnings
from statsmodels.tsa.statespace.sarimax import SARIMAX

## Modelling Methadology 

#### Overall Objective - Predict future rents For Subrubs for the next three years 

BUT - There are two glaring and *tricky** obstacles right of the bat 

* We do not have the future predictor values
* After making our predcitions we do not have a basis of comparsion for future predictons

#### Proposed methadology to tackle the modelling problem : 

1) Extrapolate 3 years *future* predictors such as income, unemployement and population using a SARImax model 

2) Train model on 2012-2021 data with rent prices as the target variable

3) Predict the future rents using the test set generated by the SARIMAX models and the already existing static information about suburbs

BUT number **2** :( 

The modelling methadology might sound good in forethought, but we need to show that the proposed methoadology is actually valid and our models do not just spit out random numbers. 

#### Steps taken to prove validity of the proposed modelling methadology 

1) We have all the TRUE data points for 2012-2019

2) Split the 2012-2019 data into the 2 sets:

* The train set : 2012-2016
* The validation (*test*)  set : 2017-2019

3) Use SARIMAX to extrapolate predictor features for 2017 - 2019 using the 2012-2016 data

4) Predict 2017-2019 rents using multiple models and compare the results with already known and true 2017-2019 results. 

5) Hopefully we will get an desirable MAE, which isnt too extreme

6) If we achive acceptable results proceed to apply this method to predict *actual* future data 

## Step 0 - Preparing Dataset in a manner suitable for modelling 

In [2]:
df = pd.read_csv('../data/curated/Final_Merged_Dataset.csv')

unwanted_years = [str(i) for i in range(2000, 2012)]

for years in unwanted_years:
    df = df[df.columns.drop(list(df.filter(regex=years)))]
    
immigration_columns = ['cost_text', 'bedrooms', 'bathrooms', 'parking_spaces' ,'2016_immigration',
 '2017_immigration',
 '2018_immigration',
 '2019_immigration',
 '2020_immigration',
 '2021_immigration']

df = df.drop(immigration_columns, axis = 1 )
df = df.groupby('suburb', as_index = False).mean()

df

,suburb,nearest_hospital,nearest_train_station,distance_to_cbd,nearest_school,2012_median_rent,2013_median_rent,2014_median_rent,2015_median_rent,2016_median_rent,...,2012_unemployment_rate,2013_unemployment_rate,2014_unemployment_rate,2015_unemployment_rate,2016_unemployment_rate,2017_unemployment_rate,2018_unemployment_rate,2019_unemployment_rate,2020_unemployment_rate,2021_unemployment_rate
0,abbotsford,2.934439,1.489185,4.443090,1.067517,450.0,470.0,425.0,440.0,450.0,...,5.3,5.5,5.4,4.5,3.4,4.1,2.9,3.1,4.7,4.5
1,albert park,44.218611,11.892911,5.411578,0.473433,470.0,460.0,480.0,500.0,520.0,...,2.3,2.7,3.2,2.4,2.2,2.1,1.6,2.1,3.6,2.7
2,alfredton,4.955048,6.510578,121.322730,1.698581,270.0,280.0,280.0,285.0,290.0,...,2.8,2.2,2.6,2.6,2.4,2.0,2.2,1.5,3.4,2.2
3,alphington,3.177273,1.260436,9.481345,1.014145,330.0,350.0,350.0,345.0,375.0,...,4.4,5.9,5.7,5.1,4.2,4.7,3.8,4.2,5.8,4.9
4,altona,1.313035,0.931396,18.746987,0.597719,330.0,340.0,350.0,360.0,375.0,...,4.7,4.5,4.7,5.0,5.2,5.7,4.3,3.5,5.0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,wendouree,2.355989,2.664463,120.395226,0.755221,270.0,280.0,280.0,285.0,290.0,...,8.9,7.3,8.9,7.7,8.1,6.2,6.9,6.0,9.0,6.2
149,west footscray,2.163598,1.444609,10.739275,1.967266,320.0,320.0,330.0,340.0,360.0,...,6.6,6.2,6.4,6.7,6.7,7.4,5.6,5.2,7.7,5.4
150,williamstown,1.569507,0.901973,14.134295,0.712710,440.0,450.0,440.0,461.0,490.0,...,3.4,3.0,3.7,3.4,3.6,3.9,3.0,2.6,4.2,2.7
151,wodonga,2.278371,4.705043,312.975948,1.161629,265.0,270.0,285.0,290.0,290.0,...,7.0,7.1,6.4,7.5,6.6,4.3,4.8,5.2,4.2,3.9


In [3]:
median_rents = ['suburb', '2012_median_rent', '2013_median_rent', '2014_median_rent', 
                 '2015_median_rent', '2016_median_rent', '2017_median_rent',  
                 '2018_median_rent', '2019_median_rent', '2020_median_rent', 
                 '2021_median_rent']

incomes = ['suburb', '2012_annual_income', '2013_annual_income', '2014_annual_income',
           '2015_annual_income', '2016_annual_income', '2017_annual_income',
           '2018_annual_income', '2019_annual_income', '2020_annual_income', 
           '2021_annual_income']

populations = ['suburb', 'population_2012', 'population_2013', 'population_2014', 
               'population_2015', 'population_2016', 'population_2017', 
               'population_2018', 'population_2019', 'population_2020', 
               'population_2021']

population_density = ['suburb', 'population_density_2012', 'population_density_2013', 'population_density_2014',
                 'population_density_2015', 'population_density_2016',
                 'population_density_2017', 'population_density_2018',
                 'population_density_2019', 'population_density_2020',
                 'population_density_2021']

unemployement = ['suburb', '2012_unemployment_rate', '2013_unemployment_rate', '2014_unemployment_rate',
                 '2015_unemployment_rate', '2016_unemployment_rate', '2017_unemployment_rate', 
                 '2018_unemployment_rate', '2019_unemployment_rate', '2020_unemployment_rate',
                 '2021_unemployment_rate']

In [4]:
melt_rents = pd.melt(df[median_rents], id_vars = 'suburb')
melt_rents['year'] = pd.to_numeric(melt_rents['variable'].str[0:4]) 
melt_rents = melt_rents.drop('variable', axis = 1) 
melt_rents = melt_rents.rename(columns = {'value' : 'median_rent'})

all_rents = melt_rents['median_rent']

melt_rents

,suburb,median_rent,year
0,abbotsford,450.000000,2012
1,albert park,470.000000,2012
2,alfredton,270.000000,2012
3,alphington,330.000000,2012
4,altona,330.000000,2012
...,...,...,...
1525,wendouree,357.208581,2021
1526,west footscray,387.983260,2021
1527,williamstown,516.017635,2021
1528,wodonga,329.083134,2021


In [5]:
melt_income = pd.melt(df[incomes], id_vars = 'suburb')
melt_income['year'] = pd.to_numeric(melt_income['variable'].str[0:4]) 
melt_income = melt_income.drop('variable', axis = 1) 
melt_income = melt_income.rename(columns = {'value' : 'mean_income'})

all_incomes = melt_income['mean_income']

melt_income

,suburb,mean_income,year
0,abbotsford,65938.000000,2012
1,albert park,107915.000000,2012
2,alfredton,52993.000000,2012
3,alphington,68272.000000,2012
4,altona,56677.000000,2012
...,...,...,...
1525,wendouree,50455.929203,2021
1526,west footscray,60293.192444,2021
1527,williamstown,91634.348140,2021
1528,wodonga,54696.818254,2021


In [6]:
melt_population = pd.melt(df[populations], id_vars = 'suburb')
melt_population['year'] = pd.to_numeric(melt_population['variable'].str[11:]) 
melt_population = melt_population.drop('variable', axis = 1) 
melt_population = melt_population.rename(columns = {'value' : 'population'})

all_populations = melt_population['population']

melt_population

,suburb,population,year
0,abbotsford,5544.0,2012
1,albert park,15142.0,2012
2,alfredton,9060.0,2012
3,alphington,8706.0,2012
4,altona,12361.0,2012
...,...,...,...
1525,wendouree,15216.0,2021
1526,west footscray,12800.0,2021
1527,williamstown,17065.0,2021
1528,wodonga,27569.0,2021


In [7]:
melt_population_density = pd.melt(df[population_density], id_vars = 'suburb')
melt_population_density['year'] = pd.to_numeric(melt_population_density['variable'].str[19:]) 
melt_population_density = melt_population_density.drop('variable', axis = 1) 
melt_population_density = melt_population_density.rename(columns = {'value' : 'population_density'})

all_population_densitu = melt_population_density['population_density']

melt_population_density

,suburb,population_density,year
0,abbotsford,3185.291583,2012
1,albert park,3239.138340,2012
2,alfredton,171.880306,2012
3,alphington,3017.363879,2012
4,altona,698.965773,2012
...,...,...,...
1525,wendouree,225.177363,2021
1526,west footscray,2140.217700,2021
1527,williamstown,2322.938078,2021
1528,wodonga,98.879859,2021


In [8]:
melt_unemployement = pd.melt(df[unemployement], id_vars = 'suburb')
melt_unemployement['year'] = pd.to_numeric(melt_unemployement['variable'].str[0:4]) 
melt_unemployement = melt_unemployement.drop('variable', axis = 1) 
melt_unemployement = melt_unemployement.rename(columns = {'value' : 'unemployement'})

all_unemployment = melt_unemployement['unemployement']

melt_unemployement

,suburb,unemployement,year
0,abbotsford,5.3,2012
1,albert park,2.3,2012
2,alfredton,2.8,2012
3,alphington,4.4,2012
4,altona,4.7,2012
...,...,...,...
1525,wendouree,6.2,2021
1526,west footscray,5.4,2021
1527,williamstown,2.7,2021
1528,wodonga,3.9,2021


In [9]:
melted_dfs = [melt_rents, melt_income, melt_population, melt_population_density, melt_unemployement ]
modelling_dataframe = pd.concat(melted_dfs, join='inner', axis=1)
modelling_dataframe = modelling_dataframe.loc[:,~modelling_dataframe.columns.duplicated()].copy()

train_modelling_dataframe = modelling_dataframe[modelling_dataframe['year'] <= 2016].reset_index(drop = True)
test_modelling_dataframe =  modelling_dataframe[(modelling_dataframe['year']>=2017) & 
                                                ( modelling_dataframe['year']<=2019)].reset_index(drop = True)

## Step 1 - Use SARIMAX to extrapolate 2017, 2018 and 2019 predictors 

* Use ARMA to determine median_rent, mean_income, population, population_density and unemployement for all suburbs , for the years 2022, 2023, 2024


In [10]:
def arma_preds(attribute):
    
    sa2_list = [ i for i in train_modelling_dataframe['suburb'].unique()]
    full_predictions = pd.DataFrame(columns = ['suburb', attribute , 'year'])
    
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        for i in sa2_list:

            curr_suburb = train_modelling_dataframe[train_modelling_dataframe['suburb'] == i]

            from statsmodels.tsa.statespace.sarimax import SARIMAX

            y = curr_suburb[attribute]

            SARIMAXmodel = SARIMAX(y, order = (0, 2, 1))
            SARIMAXmodel = SARIMAXmodel.fit(disp=0)

            y_pred = SARIMAXmodel.get_forecast(3)
            
            y_pred_df = y_pred.conf_int(alpha = 0.05) 
            y_pred_df[attribute] = SARIMAXmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
            y_pred_df["year"] = [2017, 2018, 2019]
            y_pred_df["suburb"] = i 
            
            if attribute == 'mean_income':
                y_pred_df[attribute] = y_pred_df['upper mean_income']

            full_predictions = pd.concat([full_predictions, y_pred_df[['suburb', attribute , 'year']]])
            
        return full_predictions.reset_index(drop = True)

In [11]:
future_incomes = arma_preds('mean_income')
future_population = arma_preds('population')
future_popden = arma_preds('population_density')
future_unemployment = arma_preds('unemployement')

In [12]:
future_dfs = [future_incomes, future_population, future_popden, future_unemployment]
future_dataframe = pd.concat(future_dfs, join='inner', axis=1)
future_dataframe = future_dataframe.loc[:,~future_dataframe.columns.duplicated()].copy()

future_dataframe

,suburb,mean_income,year,population,population_density,unemployement
0,abbotsford,78239.601125,2017,9561.673841,5490.157933,2.400043
1,abbotsford,85030.747427,2018,10353.347681,5941.533907,1.400085
2,abbotsford,92694.744109,2019,11145.021522,6392.909881,0.400128
3,albert park,123878.157721,2017,16896.147957,3610.324846,1.998926
4,albert park,133827.171976,2018,17302.295914,3693.150602,1.797852
...,...,...,...,...,...,...
454,wodonga,57979.702007,2018,26533.108607,95.150650,6.399963
455,wodonga,60811.493034,2019,27340.162910,98.038333,6.299945
456,yarraville,79166.503615,2017,15936.038353,2839.207027,3.650265
457,yarraville,81952.282370,2018,16216.076706,2892.549788,3.500530


In [13]:
train_modelling_dataframe = train_modelling_dataframe.merge(df[['suburb', 'nearest_hospital', 'nearest_train_station',
       'distance_to_cbd', 'nearest_school']], on = 'suburb', how = 'left' )

future_dataframe = future_dataframe.merge(df[['suburb', 'nearest_hospital', 'nearest_train_station',
       'distance_to_cbd', 'nearest_school']], on = 'suburb', how = 'left' )

display(modelling_dataframe.head(10))
display(future_dataframe.head(10))

,suburb,median_rent,year,mean_income,population,population_density,unemployement
0,abbotsford,450.0,2012,65938.0,5544.0,3185.291583,5.3
1,albert park,470.0,2012,107915.0,15142.0,3239.138340,2.3
2,alfredton,270.0,2012,52993.0,9060.0,171.880306,2.8
3,alphington,330.0,2012,68272.0,8706.0,3017.363879,4.4
4,altona,330.0,2012,56677.0,12361.0,698.965773,4.7
5,armadale,385.0,2012,99631.0,9308.0,4262.880696,2.6
6,ascot vale,390.0,2012,66592.0,14309.0,3730.090456,7.3
7,ashburton,395.0,2012,76336.0,7912.0,2789.254742,4.8
8,bairnsdale,260.0,2012,44563.0,13680.0,88.251517,5.8
9,balwyn,460.0,2012,82777.0,16319.0,2921.933751,3.5


,suburb,mean_income,year,population,population_density,unemployement,nearest_hospital,nearest_train_station,distance_to_cbd,nearest_school
0,abbotsford,78239.601125,2017,9561.673841,5490.157933,2.400043,2.934439,1.489185,4.443090,1.067517
1,abbotsford,85030.747427,2018,10353.347681,5941.533907,1.400085,2.934439,1.489185,4.443090,1.067517
2,abbotsford,92694.744109,2019,11145.021522,6392.909881,0.400128,2.934439,1.489185,4.443090,1.067517
3,albert park,123878.157721,2017,16896.147957,3610.324846,1.998926,44.218611,11.892911,5.411578,0.473433
4,albert park,133827.171976,2018,17302.295914,3693.150602,1.797852,44.218611,11.892911,5.411578,0.473433
5,albert park,145842.344810,2019,17708.443871,3775.976357,1.596778,44.218611,11.892911,5.411578,0.473433
6,alfredton,63183.150880,2017,12707.908189,240.992918,2.300012,4.955048,6.510578,121.322730,1.698581
7,alfredton,66535.763201,2018,13563.816378,257.137560,2.200023,4.955048,6.510578,121.322730,1.698581
8,alfredton,70123.619184,2019,14419.724566,273.282201,2.100035,4.955048,6.510578,121.322730,1.698581
9,alphington,80553.583625,2017,9783.351198,3389.437070,3.132991,3.177273,1.260436,9.481345,1.014145


#### We now have both our train and test set ready ! - Need to Transform Appropiately and Fit Models 

## Step 1.1 - Applying Appropriate Transformations and Creating Matrices Suitable for The Models 

In [14]:
X_train = train_modelling_dataframe.drop(['median_rent', 'year'], axis = 1)
X_train = pd.get_dummies(X_train, columns=['suburb'])

y_train = train_modelling_dataframe['median_rent']

X_train.head()

,mean_income,population,population_density,unemployement,nearest_hospital,nearest_train_station,distance_to_cbd,nearest_school,suburb_abbotsford,suburb_albert park,...,suburb_tullamarine,suburb_vermont,suburb_wangaratta,suburb_wantirna,suburb_warragul,suburb_wendouree,suburb_west footscray,suburb_williamstown,suburb_wodonga,suburb_yarraville
0,65938.0,5544.0,3185.291583,5.3,2.934439,1.489185,4.443090,1.067517,1,0,...,0,0,0,0,0,0,0,0,0,0
1,107915.0,15142.0,3239.138340,2.3,44.218611,11.892911,5.411578,0.473433,0,1,...,0,0,0,0,0,0,0,0,0,0
2,52993.0,9060.0,171.880306,2.8,4.955048,6.510578,121.322730,1.698581,0,0,...,0,0,0,0,0,0,0,0,0,0
3,68272.0,8706.0,3017.363879,4.4,3.177273,1.260436,9.481345,1.014145,0,0,...,0,0,0,0,0,0,0,0,0,0
4,56677.0,12361.0,698.965773,4.7,1.313035,0.931396,18.746987,0.597719,0,0,...,0,0,0,0,0,0,0,0,0,0


## Step 1.2 - Create Test Set Dataframes 

In [15]:
X_test_2017 = future_dataframe[future_dataframe['year'] == 2017].reset_index(drop = True)
X_test_2017 = X_test_2017.drop(['year'], axis = 1)
suburbs_list = X_test_2017['suburb']
X_test_2017 = pd.get_dummies(X_test_2017, columns=['suburb'])

In [16]:
X_test_2018 = future_dataframe[future_dataframe['year'] == 2018].reset_index(drop = True)
X_test_2018 = X_test_2018.drop(['year'], axis = 1)
X_test_2018 = pd.get_dummies(X_test_2018, columns=['suburb'])

X_test_2018.head()

,mean_income,population,population_density,unemployement,nearest_hospital,nearest_train_station,distance_to_cbd,nearest_school,suburb_abbotsford,suburb_albert park,...,suburb_tullamarine,suburb_vermont,suburb_wangaratta,suburb_wantirna,suburb_warragul,suburb_wendouree,suburb_west footscray,suburb_williamstown,suburb_wodonga,suburb_yarraville
0,85030.747427,10353.347681,5941.533907,1.400085,2.934439,1.489185,4.443090,1.067517,1,0,...,0,0,0,0,0,0,0,0,0,0
1,133827.171976,17302.295914,3693.150602,1.797852,44.218611,11.892911,5.411578,0.473433,0,1,...,0,0,0,0,0,0,0,0,0,0
2,66535.763201,13563.816378,257.137560,2.200023,4.955048,6.510578,121.322730,1.698581,0,0,...,0,0,0,0,0,0,0,0,0,0
3,85342.490587,10132.702395,3509.196811,2.065982,3.177273,1.260436,9.481345,1.014145,0,0,...,0,0,0,0,0,0,0,0,0,0
4,72856.939817,13663.994338,772.918591,5.549820,1.313035,0.931396,18.746987,0.597719,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X_test_2019 = future_dataframe[future_dataframe['year'] == 2019].reset_index(drop = True)
X_test_2019 = X_test_2019.drop(['year'], axis = 1)
X_test_2019 = pd.get_dummies(X_test_2019, columns=['suburb'])

X_test_2019.head()

,mean_income,population,population_density,unemployement,nearest_hospital,nearest_train_station,distance_to_cbd,nearest_school,suburb_abbotsford,suburb_albert park,...,suburb_tullamarine,suburb_vermont,suburb_wangaratta,suburb_wantirna,suburb_warragul,suburb_wendouree,suburb_west footscray,suburb_williamstown,suburb_wodonga,suburb_yarraville
0,92694.744109,11145.021522,6392.909881,0.400128,2.934439,1.489185,4.443090,1.067517,1,0,...,0,0,0,0,0,0,0,0,0,0
1,145842.344810,17708.443871,3775.976357,1.596778,44.218611,11.892911,5.411578,0.473433,0,1,...,0,0,0,0,0,0,0,0,0,0
2,70123.619184,14419.724566,273.282201,2.100035,4.955048,6.510578,121.322730,1.698581,0,0,...,0,0,0,0,0,0,0,0,0,0
3,90942.364709,10482.053593,3628.956552,0.998973,3.177273,1.260436,9.481345,1.014145,0,0,...,0,0,0,0,0,0,0,0,0,0
4,77304.877222,13892.991507,786.004292,5.724731,1.313035,0.931396,18.746987,0.597719,0,0,...,0,0,0,0,0,0,0,0,0,0


## Step 3 - Moment of truth - Using proposed methadology to 'predict' 2017 - 2019 rents and compare to true values 

#### Support Vector Machine

In [18]:
from sklearn.metrics import mean_absolute_error as mae, r2_score

future_true = [i for i in test_modelling_dataframe['median_rent']]

In [19]:
from sklearn.svm import SVR

sv_reg_regressor = SVR()
sv_reg_regressor.fit(X_train, y_train)

y_pred_2017 = sv_reg_regressor.predict(X_test_2017)
y_pred_2018 = sv_reg_regressor.predict(X_test_2018)
y_pred_2019 = sv_reg_regressor.predict(X_test_2019)

sv_reg_predictions = pd.DataFrame(columns = ['Suburb', '2017 Predicted Rent', 
                                                '2018 Predicted Rent', '2019 Predicted Rent'])

sv_all_preds =  np.concatenate([y_pred_2017, y_pred_2018, y_pred_2019], axis=0)

mae(future_true, sv_all_preds )

60.3916710703771

#### XGboost Regression 

In [20]:
from xgboost.sklearn import XGBRegressor

xgboost_regressor = XGBRegressor()
xgboost_regressor.fit(X_train, y_train)

y_pred_2017 = xgboost_regressor.predict(X_test_2017)
y_pred_2018 = xgboost_regressor.predict(X_test_2018)
y_pred_2019 = xgboost_regressor.predict(X_test_2019)

xg_all_preds =  np.concatenate([y_pred_2017, y_pred_2018, y_pred_2019], axis=0)

mae(future_true, xg_all_preds )

25.728620142718547

#### Elastic Net

In [21]:
from sklearn.linear_model import ElasticNet


elasticnet_regressor = ElasticNet()
elasticnet_regressor.fit(X_train, y_train)

y_pred_2017 = elasticnet_regressor.predict(X_test_2017)
y_pred_2018 = elasticnet_regressor.predict(X_test_2018)
y_pred_2019 = elasticnet_regressor.predict(X_test_2019)

en_all_preds =  np.concatenate([y_pred_2017, y_pred_2018, y_pred_2019], axis=0)
print(mae(future_true, en_all_preds))

39.97952938896978


In [22]:
mae(future_true, en_all_preds )

39.97952938896978

#### Decision Tree Regressor 

In [23]:
from sklearn.tree import DecisionTreeRegressor

tree_regressor = DecisionTreeRegressor()
tree_regressor.fit(X_train, y_train)

y_pred_2017 = tree_regressor.predict(X_test_2017)
y_pred_2018 = tree_regressor.predict(X_test_2018)
y_pred_2019 = tree_regressor.predict(X_test_2019)

dt_all_preds =  np.concatenate([y_pred_2017, y_pred_2018, y_pred_2019], axis=0)
mae(future_true, dt_all_preds )

33.930283224400874

#### Random Forest Regression

In [24]:
from sklearn.ensemble import RandomForestRegressor

randomforest_regressor = RandomForestRegressor()
randomforest_regressor.fit(X_train, y_train)

y_pred_2017 = randomforest_regressor.predict(X_test_2017)
y_pred_2018 = randomforest_regressor.predict(X_test_2018)
y_pred_2019 = randomforest_regressor.predict(X_test_2019)

rf_all_preds =  np.concatenate([y_pred_2017, y_pred_2018, y_pred_2019], axis=0)

mae(future_true, rf_all_preds )

29.774738562091503

### We will use XGboost and random forest regression (ensemble) for our future prediction as they have the lowest MAE and highest r2 scores 